# CartPole-v0

> reference: 

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. 

A `reward` of +1 is provided for every timestep that the pole remains upright. The episode ends when `the pole is more than 15 degrees from vertical`, or `the cart moves more than 2.4 units from the center`.

In [1]:
import gym, threading, random, itertools, time
import tensorflow as tf
from collections import deque
import numpy as np


random.seed(42)

In [2]:
class Brain():
    def __init__(self, state_size, action_size):
        self._state_size = state_size
        self._action_size = action_size     
        self.gamma = 0.95    # discount rate
        self._learning_rate = 0.001
        
        self.value_size = 1        
        self.actor = self._build_actor()
        self.critic = self._build_critic()
        
        # tensorflow
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def _build_actor(my, hidden_size=24):
        class actorModel():
            def __init__(self):
                self.x = tf.placeholder(tf.float32, [None, my._state_size])
                nn = tf.layers.dense(self.x, hidden_size, activation=tf.nn.relu)
                nn = tf.layers.dense(nn, hidden_size, activation=tf.nn.relu)
                all_act = tf.layers.dense(nn, my._action_size)#, activation=tf.nn.linear)
                self.all_act_prob = tf.nn.softmax(all_act, name='act_prob')
        
                # op to sample an action
                self._sample = tf.reshape(tf.multinomial(self.all_act_prob, 1), [])

                self.advantages = tf.placeholder(tf.float32, [None, my._action_size]) 
                self.loss = -tf.reduce_sum(tf.multiply(self.all_act_prob, self.advantages))

                # update
                optimizer = tf.train.AdamOptimizer(my._learning_rate)
                self.train_op = optimizer.minimize(self.loss)
                
                # Summary for TensorBoard
                tf.summary.scalar('loss', self.loss)
                self.summary = tf.summary.merge_all()    # summary
                
        return actorModel()
    
    # critic: state is input and value of state is output of model
    def _build_critic(my, hidden_size=24):
        class criticModel():
            def __init__(self):
                self.x = tf.placeholder(tf.float32, [None, my._state_size])
                nn = tf.layers.dense(self.x, hidden_size, activation=tf.nn.relu)
                nn = tf.layers.dense(nn, hidden_size, activation=tf.nn.relu)
                self.Q = tf.layers.dense(nn, my.value_size)#, activation=tf.nn.linear)
        
                # Loss
                self.targetQ = tf.placeholder(tf.float32, [None, my.value_size])
                self.loss = tf.reduce_mean((self.Q - self.targetQ)**2)
                
                # Summary for TensorBoard
                tf.summary.scalar('loss', self.loss)
                self.summary = tf.summary.merge_all()    # summary
                
                # Training
                self.train_op = tf.train.AdamOptimizer(my._learning_rate).minimize(self.loss)   
                
        return criticModel()



In [3]:
class A2CAgent():
    def __init__(self):      
        self.memory = deque(maxlen=2000)
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
      
    def train_model(self, batch_size=20):
        def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
                yield list(itertools.islice(iterable, ndx, min(ndx + n, l)))
                
                
        batch_data = list(batch(self.memory, batch_size))
        selected_batch = random.sample(batch_data, 1)
        
        for state, action, reward, next_state, done in selected_batch[0]:
            
            target = np.zeros((1, brain.value_size))
            advantages = np.zeros((1, brain._action_size))
            
            value = brain.sess.run(brain.critic.Q, feed_dict={brain.critic.x: [state]}) # self.critic.predict(state)[0]
            next_value = brain.sess.run(brain.critic.Q, feed_dict={brain.critic.x: [next_state]}) # self.critic.predict(next_state)[0]
            
            if done:
                advantages[0][action] = reward - value
                target[0][0] = reward
            else:
                advantages[0][action] = reward + brain.gamma * (next_value) - value
                target[0][0] = reward + brain.gamma * next_value
            
            feed_dict_actor = {
                brain.actor.x: [state],
                brain.actor.advantages: advantages,
            }
            feed_dict_critic = {
                brain.critic.x: [state], 
                brain.critic.targetQ: target,
            }
            _, actor_loss = brain.sess.run([brain.actor.train_op, brain.actor.loss], feed_dict=feed_dict_actor)
#             summary = brain.sess.run([brain.actor.summary], feed_dict=feed_dict_actor)
            _, critic_loss = brain.sess.run([brain.critic.train_op, brain.critic.loss], feed_dict=feed_dict_critic)
#             summary = brain.sess.run([brain.critic.summary], feed_dict=feed_dict_critic)


    # using the output of policy network, pick action stochastically
    def takeAction(self, state):
        return brain.sess.run(brain.actor._sample, feed_dict={brain.actor.x: [state]})
    

In [4]:
class Environment(threading.Thread):
    stop_signal = False
    ENV = 'CartPole-v0'
    THREAD_DELAY = 0.001
    
    def __init__(self, render=False):
        threading.Thread.__init__(self)

        self.render = render
        self.env = gym.make(self.ENV)
        self.agent = A2CAgent()
        
        self.NUM_STATE = self.env.observation_space.shape[0]
        self.NUM_ACTIONS = self.env.action_space.n

        self.episodes = 0
        self.shows = 20

    def runEpisode(self):

        state = self.env.reset()
        Rewards = 0

        while True: 
            time.sleep(self.THREAD_DELAY) # yield

            if self.render: self.env.render()

            action = self.agent.takeAction(state)
            next_state, reward, done, _ = self.env.step(action)

            self.agent.remember(state, action, reward, next_state, done)
            state = next_state
            Rewards += reward

            if done or self.stop_signal: break

        self.agent.train_model(32)
        
        if Rewards >= 200:
            self.stop()
            
        print("Episode {}, Reward: {}".format(self.episodes, Rewards))
        self.episodes += 1
        
            
    def run(self):
        while not self.stop_signal:
            self.runEpisode()

    def stop(self):
        self.stop_signal = True

In [5]:
env_test = Environment(render=True)
RUN_TIME = 30
THREADS = 4

brain = Brain(env_test.NUM_STATE, env_test.NUM_ACTIONS)    # brain is global in A3C

envs = [Environment() for i in range(THREADS)]

for e in envs:
    e.start()

time.sleep(RUN_TIME)

for e in envs:
    e.stop()
for e in envs:
    e.join()

print("Training finished")
env_test.run()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 0, Reward: 17.0
Episode 0, Reward: 21.0
Episode 0, Reward: 24.0
Episode 0, Reward: 44.0
Episode 1, Reward: 20.0
Episode 1, Reward: 29.0
Episode 1, Reward: 22.0
Episode 1, Reward: 18.0
Episode 2, Reward: 12.0
Episode 2, Reward: 19.0
Episode 2, Reward: 13.0
Episode 3, Reward: 19.0
Episode 2, Reward: 23.0
Episode 3, Reward: 12.0Episode 3, Reward: 12.0

Episode 4, Reward: 10.0
Episode 3, Reward: 10.0
Episode 4, Reward: 13.0
Episode 5, Reward: 14.0
Episode 4, Reward: 15.0
Episode 5, Reward: 17.0
E

Episode 6, Reward: 51.0
Episode 7, Reward: 95.0
Episode 8, Reward: 98.0
Episode 9, Reward: 44.0
Episode 10, Reward: 200.0
